In [2]:
import math
import torch
import numpy as np
# import gpytorch
import pandas as pd
import os
import pickle
import shutil

import sklearn
from sklearn.model_selection import KFold

import imblearn as imb
# print("imblearn version: ",imblearn.__version__)
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import itertools

from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score

from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from rdkit import Chem
from rdkit.Chem import Draw
import sys
sys.path.append('../')
import utils

from sklearn.model_selection import GridSearchCV
from VisUtils import *
from RF_GSCV import *
from sklearn.model_selection import KFold
from split_data import *
from matplotlib import pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
def scale_data(train_x_df, train_y_df, test_x_df, test_y_df): 
    x_df = pd.concat([train_x_df, test_x_df])
    scaling=StandardScaler()
    scaling.fit(x_df)
    Scaled_data=scaling.transform(x_df)
    train_x = scaling.transform(train_x_df)
    test_x = scaling.transform(test_x_df) 
    
    train_y = train_y_df.to_numpy().flatten()
    test_y = test_y_df.to_numpy().flatten()
    return train_x, train_y, test_x, test_y
    

In [3]:
# original splits
split_path = '../../../../data/NEK_data_4Berkeley/NEK2/'
ogtrain_x_df = pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_trainX.csv")
ogtrain_y_df= pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_trainY.csv")
ogtest_x_df= pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_testX.csv")
ogtest_y_df= pd.read_csv(split_path+"/NEK2_inhibition_random_fold1_testY.csv")

# Split data 

Take in MOE descriptor dataset from rdkit smiles strings, split majority and minority and add assign folds to make balanced across folds 

In [ ]:
# data_path = '../../../../data/datasets/'
# # this is rdkit smiles strings, pct binding, target NEK, compound id, active yes/no
# nek2inhib_1uM_50pct = 'NEK2_1_uM_min_50_pct_inhibition.csv'
# inhib_1uM50pct = pd.read_csv(data_path+nek2inhib_1uM_50pct)

In [6]:
# data_path_feat = '../../../../data/datasets/scaled_descriptors/'
# nek2binhib_MOE = 'NEK2_1_uM_min_50_pct_inhibition_with_moe_descriptors.csv'
# MOE_dataset = pd.read_csv(data_path_feat+nek2binhib_MOE)
# split_path = 'nek2_inhibfeaturesplits_jaycee'
# all_fold_MOE = make_splits(MOE_dataset, data_path_feat,'NEK2inhib')

In [ ]:
# all_fold_MOE['compound_id'].value_counts()
# all_fold_MOE['base_rdkit_smiles'].value_counts()

In [ ]:
all_fold_MOE.shape

# Split into train/test splits based on fold 
Here, fold1 is the testset 

In [7]:
# nek2jp_path = '../../../../data/nek2_featuresplits_jaycee/'
# trainxtorch, trainytorch, testxtorch, testytorch = get_datasplits(nek2jp_path, 'nek2inhib', all_fold_MOE)
# all_folds = pd.read_csv

In [8]:
trainxtorch.shape, trainytorch.shape, testxtorch.shape, testytorch.shape

NameError: name 'trainxtorch' is not defined

In [9]:
train_x, train_y, test_x, test_y = scale_data(pd.DataFrame(trainxtorch), pd.DataFrame(trainytorch),
                                              pd.DataFrame(testxtorch), pd.DataFrame(testytorch))

trainxdf = pd.DataFrame(train_x)
trainydf = pd.DataFrame(train_y)
testxdf = pd.DataFrame(test_x)
testydf = pd.DataFrame(test_y)
split_path = '../../../../data/NEK_data_4Berkeley/NEK2/inhib/'
trainxdf.to_csv(split_path+'inhib_train_x_scaledoriginal.csv',index=False, header=False)
trainydf.to_csv(split_path+'inhib_train_y_scaledoriginal.csv',index=False, header=False)
testxdf.to_csv(split_path+'inhib_test_x_scaledoriginal.csv',index=False, header=False)
testydf.to_csv(split_path+'inhib_test_y_scaledoriginal.csv',index=False, header=False)

NameError: name 'scale_data' is not defined

In [ ]:
testydf.value_counts()

In [ ]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

In [ ]:
split_path = '../../../../data/NEK_data_4Berkeley/NEK2/inhib/'
train_x_df_temp = pd.read_csv(split_path+'inhib_train_x_scaledoriginal.csv',header=None)
train_y_df_temp = pd.read_csv(split_path+'inhib_train_y_scaledoriginal.csv',header=None)
test_x_df_temp = pd.read_csv(split_path+'inhib_test_x_scaledoriginal.csv',header=None)
test_y_df_temp = pd.read_csv(split_path+'inhib_test_y_scaledoriginal.csv',header=None)

train_x_df_temp.shape, train_y_df_temp.shape, test_x_df_temp.shape, test_y_df_temp.shape

In [ ]:
smile_prop_df = pd.read_csv('../../../../data/datasets/NEK_data_4Berkeley/NEK2/NEK2_1_uM_min_50_pct_inhibition_5fold_random_imbalanced.csv')
trues1 = smile_prop_df[smile_prop_df['fold']=='fold1']['active'].value_counts()
trues1.value_counts()
trues1
smile_prop_df['active'].value_counts()

# Normalizing


In [10]:
smile_prop_df = pd.read_csv('../../../../data/datasets/NEK_data_4Berkeley/NEK2/NEK2_1_uM_min_50_pct_inhibition_5fold_random_imbalanced.csv')
# smile prop train will be normalized feats
smile_prop_train = smile_prop_df[smile_prop_df['fold'] != 'fold1'].drop(columns=['fold'])
smile_prop_test = smile_prop_df[smile_prop_df['fold'] == 'fold1'].drop(columns=['fold'])
mol_train = []
for i in smile_prop_train.index:
    m = Chem.MolFromSmiles(smile_prop_train.loc[i, 'base_rdkit_smiles'])
    if m:
        m.SetProp("_Name", smile_prop_train.loc[i, 'compound_id'])
        m.SetProp("Active", str(smile_prop_train.loc[i, 'active']))
        mol_train.append(utils.normalize(m))

mol_test = []
for i in smile_prop_test.index:
    m = Chem.MolFromSmiles(smile_prop_test.loc[i, 'base_rdkit_smiles'])
    if m:
        m.SetProp("_Name", smile_prop_test.loc[i, 'compound_id'])
        m.SetProp("Active", str(smile_prop_test.loc[i, 'active']))
        mol_test.append(utils.normalize(m))
# normalized feats from smile_prop_train
X_existing_train = smile_prop_train.drop(columns=['compound_id', 'base_rdkit_smiles', 'active'])
X_existing_test = smile_prop_test.drop(columns=['compound_id', 'base_rdkit_smiles', 'active'])
y_train = smile_prop_train['active']
y_test = smile_prop_test['active']
# print(y_test.value_counts())
# train_x.shape, train_y.shape, test_x.shape, test_y.shape

# generate fingeprints: Morgan fingerprint with radius 2
fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mol_train]
# convert the RDKit explicit vectors into numpy arrays
X_morgan_train = []
for fp in fps:
  arr = np.zeros((1,))
  Chem.DataStructs.ConvertToNumpyArray(fp, arr)
  X_morgan_train.append(arr)

# generate fingeprints: Morgan fingerprint with radius 2
fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mol_test]
# convert the RDKit explicit vectors into numpy arrays
X_morgan_test = []
for fp in fps:
  arr = np.zeros((1,))
  Chem.DataStructs.ConvertToNumpyArray(fp, arr)
  X_morgan_test.append(arr)

[00:00:58] Initializing MetalDisconnector
[00:00:58] Running MetalDisconnector
[00:00:58] Initializing Normalizer
[00:00:58] Running Normalizer
[00:00:58] Running LargestFragmentChooser
[00:00:58] Running Uncharger
[00:00:58] Initializing MetalDisconnector
[00:00:58] Running MetalDisconnector
[00:00:58] Initializing Normalizer
[00:00:58] Running Normalizer
[00:00:58] Initializing MetalDisconnector
[00:00:58] Running MetalDisconnector
[00:00:58] Initializing Normalizer
[00:00:58] Running Normalizer
[00:00:58] Running LargestFragmentChooser
[00:00:58] Running Uncharger
[00:00:58] Initializing MetalDisconnector
[00:00:58] Running MetalDisconnector
[00:00:58] Initializing Normalizer
[00:00:58] Running Normalizer
[00:00:58] Initializing MetalDisconnector
[00:00:58] Running MetalDisconnector
[00:00:58] Initializing Normalizer
[00:00:58] Running Normalizer
[00:00:58] Running LargestFragmentChooser
[00:00:58] Running Uncharger
[00:00:58] Removed negative charge.
[00:00:58] Initializing MetalDi

RuntimeError: Invariant Violation
	could not find atom1
	Violation occurred on line 227 in file Code/GraphMol/Canon.cpp
	Failed Expression: firstFromAtom1
	RDKIT: 2023.09.4
	BOOST: 1_78


In [ ]:
y_test_1s =np.unique( y_test, return_counts=True)
y_test_1s


In [ ]:
trainX_norm = X_existing_train.to_numpy()
# y_train = y_train.to_numpy().reshape(-1)
testX_norm = X_existing_test.to_numpy()
# y_test = y_test.to_numpy().reshape(-1)
# back to df for saving 
train_x_df = pd.DataFrame(trainX_norm) 
train_y_df = pd.DataFrame(y_train) 
test_y_df = pd.DataFrame(y_test)
test_x_df = pd.DataFrame(testX_norm)

train_x_df.to_csv(split_path+'inhib_train_x_normalized.csv', index=False, header=False)
train_y_df.to_csv(split_path+'inhib_train_y_normalized.csv', index=False, header=False) 
test_y_df.to_csv(split_path+'inhib_test_y_normalized.csv', index=False, header=False) 
test_x_df.to_csv(split_path+'inhib_test_x_normalized.csv', index=False, header=False) 

In [ ]:
split_path = '../../../../data/NEK_data_4Berkeley/NEK2/inhib/'
trainX_norm = pd.read_csv(split_path+"inhib_train_x_normalized.csv",header=None)
train_y_df= pd.read_csv(split_path+"inhib_train_y_normalized.csv",header=None)
test_x_df= pd.read_csv(split_path+"inhib_test_x_normalized.csv",header=None)
test_y_df= pd.read_csv(split_path+"inhib_test_y_normalized.csv",header=None)


trainX_norm= trainX_norm.to_numpy()
y_train = train_y_df.to_numpy().reshape(-1)
testX_norm = test_x_df.to_numpy()
test_y = test_y_df.to_numpy().reshape(-1)
trainX_norm.shape, y_train.shape, testX_norm.shape, test_y.shape

In [ ]:
test_y_df.value_counts()

# norm SMOTE

In [ ]:
# SMOTE that ^ 
oversample = SMOTE()
trainX_normSMOTE_temp, trainy_normSMOTE_temp = oversample.fit_resample(trainX_norm, y_train)
trainX_normSMOTE = trainX_normSMOTE_temp
trainy_normSMOTE = trainy_normSMOTE_temp.reshape(-1)

train_x_dfSMOTE = pd.DataFrame(trainX_normSMOTE) 
train_y_dfSMOTE = pd.DataFrame(trainy_normSMOTE.reshape(-1)) 
test_y_df = pd.DataFrame(test_y.reshape(-1))
test_x_df = pd.DataFrame(testX_norm)
train_x_dfSMOTE.shape, train_y_dfSMOTE.shape, test_y_df.shape, test_x_df.shape

# train_x_dfSMOTE.to_csv(split_path+'inhib_train_x_normSMOTE.csv', index=False, header=False)
# train_y_dfSMOTE.to_csv(split_path+'inhib_train_y_normSMOTE.csv', index=False, header=False) 
# test_y_df.to_csv(split_path+'inhib_test_y_normSMOTE.csv', index=False, header=False) 
# test_x_df.to_csv('inhib_test_x_normSMOTE.csv', index=False, header=False) 

In [ ]:
test_y_df.value_counts()

In [ ]:
tset_y_df.value_counts()

# norm UNDER

In [ ]:

undersample = RandomUnderSampler()
train_x_UNDER, train_y_UNDER = undersample.fit_resample(trainX_norm, y_train)

train_y_UNDER = train_y_UNDER.flatten()

train_x_dfUNDER = pd.DataFrame(train_x_UNDER) 
train_y_dfUNDER = pd.DataFrame(train_y_UNDER) 
test_y_df = pd.DataFrame(test_y.reshape(-1))
test_x_df = pd.DataFrame(testX_norm)

# train_x_dfUNDER.to_csv(split_path+'inhib_train_x_normUNDER.csv', index=False, header=False)
# train_y_dfUNDER.to_csv(split_path+'inhib_train_y_normUNDER.csv', index=False, header=False) 
# test_y_df.to_csv(split_path+'inhib_test_y_normUNDER.csv', index=False, header=False) 
# test_x_df.to_csv(split_path+'inhib_test_x_normUNDER.csv', index=False, header=False) 

# norm ADASYN

In [ ]:

# adasyn = ADASYN() 
# train_x_tempADASYN, train_y_tempADASYN = adasyn.fit_resample(trainX_norm, y_train)
# train_xADASYN = train_x_tempADASYN
# train_yADASYN = train_y_tempADASYN.flatten()

# train_xADASYN.shape,train_yADASYN.shape, testX_norm.shape, test_y.shape

# train_x_dfADASYN = pd.DataFrame(train_xADASYN) 
# train_y_dfADASYN = pd.DataFrame(train_yADASYN.reshape(-1)) 
# test_y_df = pd.DataFrame(test_y.reshape(-1))
# test_x_df = pd.DataFrame(testX_norm)

# train_x_dfADASYN.to_csv(split_path+'inhib_train_x_normADASYN.csv', index=False, header=False)
# train_y_dfADASYN.to_csv(split_path+'inhib_train_y_normADASYN.csv', index=False, header=False) 
# test_y_df.to_csv(split_path+'inhib_test_y_normADASYN.csv', index=False, header=False) 
# test_x_df.to_csv(split_path+'inhib_test_x_normADASYN.csv', index=False, header=False) 

# Morgan fingerprints wiht radius 2 
r=2

In [ ]:
X_morgan_train 
X_morgan_test = X_morgan_test
y_trainmorgan = smile_prop_train['active']
y_testmorgan = smile_prop_test['active']

train_x_dfmorgan = pd.DataFrame(X_morgan_train) 
train_y_dfmorgan = pd.DataFrame(y_trainmorgan) 
test_y_dfmorgan = pd.DataFrame(y_testmorgan)
test_x_dfmorgan = pd.DataFrame(X_morgan_test)

# train_x_dfmorgan.to_csv(split_path+'inhib_train_x_normMorgan2.csv', index=False, header=False)
# train_y_dfmorgan.to_csv(split_path+'inhib_train_y_normMorgan2.csv', index=False, header=False) 
# test_y_dfmorgan.to_csv(split_path+'inhib_test_y_normMorgan2.csv', index=False, header=False) 
# test_x_dfmorgan.to_csv(split_path+'inhib_test_x_normMorgan2.csv', index=False, header=False) 

In [ ]:
train_x_dfmorgan.shape, train_y_dfmorgan.shape, test_y_dfmorgan.shape, test_x_dfmorgan.shape

# Morgan fingerprints + SMOTE 
r = 2

In [ ]:
oversample = SMOTE()
trainX_Morgan_SMOTE, trainy_Morgan_SMOTE = oversample.fit_resample(X_morgan_train, y_train)
train_x_dfmorganSMOTE = pd.DataFrame(trainX_Morgan_SMOTE) 
train_y_dfmorganSMOTE = pd.DataFrame(trainy_Morgan_SMOTE) 
test_y_dfmorganSMOTE = pd.DataFrame(y_testmorgan)
test_x_dfmorganSMOTE = pd.DataFrame(X_morgan_test)

train_x_dfmorganSMOTE.to_csv(split_path+'inhib_train_x_Morgan2SMOTE.csv', index=False, header=False)
train_y_dfmorganSMOTE.to_csv(split_path+'inhib_train_y_Morgan2SMOTE.csv', index=False, header=False) 
test_y_dfmorganSMOTE.to_csv(split_path+'inhib_test_y_Morgan2SMOTE.csv', index=False, header=False) 
test_x_dfmorganSMOTE.to_csv(split_path+'inhib_test_x_Morgan2SMOTE.csv', index=False, header=False) 
train_x_dfmorganSMOTE.shape, train_y_dfmorganSMOTE.shape, test_y_dfmorganSMOTE.shape, test_x_dfmorganSMOTE.shape

In [ ]:
test_y_dfmorganSMOTE.value_counts()

# Morgan fingerprints + UNDER

In [ ]:
undersample = RandomUnderSampler()
train_x_Morgan_UNDER, train_y_Morgan_UNDER = undersample.fit_resample(X_morgan_train, y_train)

train_x_dfMorganUNDER = pd.DataFrame(train_x_Morgan_UNDER) 
train_y_dfMorganUNDER = pd.DataFrame(train_y_Morgan_UNDER) 
test_y_dfMorganUnder = pd.DataFrame(y_testmorgan)
test_x_dfMorganUnder = pd.DataFrame(X_morgan_test)

# train_x_dfMorganUNDER.to_csv(split_path+'inhib_train_x_Morgan2UNDER.csv', index=False, header=False)
# train_y_dfMorganUNDER.to_csv(split_path+'inhib_train_y_Morgan2UNDER.csv', index=False, header=False) 
# test_y_dfMorganUnder.to_csv(split_path+'inhib_test_y_Morgan2UNDER.csv', index=False, header=False) 
# test_x_dfMorganUnder.to_csv(split_path+'inhib_test_x_Morgan2UNDER.csv', index=False, header=False) 

# Morgan fingerprints + ADASYN

In [ ]:

adasyn = ADASYN() 
train_x_MorganADASYN, train_y_MorganADASYN = adasyn.fit_resample(X_morgan_train, y_train)


train_x_dfMorganADASYN = pd.DataFrame(train_x_MorganADASYN) 
train_y_dfMorganADASYN = pd.DataFrame(train_y_MorganADASYN) 
test_y_dfMorganADASYN = pd.DataFrame(y_testmorgan)
test_x_dfMorganADASYN = pd.DataFrame(X_morgan_test)

# train_x_dfMorganADASYN.to_csv(split_path+'inhib_train_x_Morgan2ADASYN.csv', index=False, header=False)
# train_y_dfMorganADASYN.to_csv(split_path+'inhib_train_y_Morgan2ADASYN.csv', index=False, header=False) 
# test_y_dfMorganADASYN.to_csv(split_path+'inhib_test_y_Morgan2ADASYN.csv', index=False, header=False) 
# test_x_dfMorganADASYN.to_csv(split_path+'inhib_test_x_Morgan2ADASYN.csv', index=False, header=False) 

In [ ]:
X_existing_train.shape, len(X_morgan_train), X_existing_test.shape, len(X_morgan_test)

# Combine MOE with Morgan fingerprints 

In [ ]:
X_train_allfeats = np.hstack((X_existing_train, X_morgan_train))
X_test_allfeats = np.hstack((X_existing_test, X_morgan_test)) # X_test_allfeats.shape (283, 2354) # now 2354 feats
# y_test and y_testmrogan == same 
train_combo = pd.DataFrame(X_train_allfeats) # train X 
test_combo = pd.DataFrame(X_test_allfeats) # test X 
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_testmorgan)
# train_combo.shape, test_combo.shape, X_existing_train.shape, len(X_morgan_train), X_train_allfeats.shape
# train_combo.to_csv(split_path+'inhib_train_x_MOEMorgan2combo.csv', index=False, header=False)
# y_train_df.to_csv(split_path+'inhib_train_y_MOEMorgan2combo.csv', index=False, header=False)
# y_test_df.to_csv(split_path+'inhib_test_y_MOEMorgan2combo.csv', index=False, header=False)
# test_combo.to_csv(split_path+'inhib_test_x_MOEMorgan2combo.csv', index=False, header=False)

In [ ]:
train_combo.shape, y_train_df.shape, test_combo.shape,y_test_df.shape

# Combo r = 2, SMOTE 

In [ ]:
oversample = SMOTE()
trainX_allSMOTE, trainy_allSMOTE = oversample.fit_resample(X_train_allfeats, y_train)
trainXdfcomboSMOTE = pd.DataFrame(trainX_allSMOTE)
trainydfcomboSMOTE = pd.DataFrame(trainy_allSMOTE)
test_x_comboSMOTE = pd.DataFrame(X_test_allfeats)

# trainXdfcomboSMOTE.to_csv(split_path+'inhib_train_x_Morgan2comboSMOTE.csv', index=False, header=False)
# trainydfcomboSMOTE.to_csv(split_path+'inhib_train_y_Morgan2comboSMOTE.csv', index=False, header=False)
# y_test_df.to_csv(split_path+'inhib_test_y_Morgan2comboSMOTE.csv', index=False, header=False)
# test_x_comboSMOTE.to_csv(split_path+'inhib_test_x_Morgan2comboSMOTE.csv', index=False, header=False)
trainXdfcomboSMOTE.shape, trainydfcomboSMOTE.shape, test_x_comboSMOTE.shape, y_test.shape

# Combo r=2, UNDER

In [ ]:
undersampler = RandomUnderSampler()
trainX_allUNDER, trainy_allUNDER = undersampler.fit_resample(X_train_allfeats, y_train) 
trainXallUNDERdf = pd.DataFrame(trainX_allUNDER)
trainyallUNDERdf = pd.DataFrame(trainy_allUNDER)
testxdf = pd.DataFrame(X_test_allfeats)
testydf = pd.DataFrame(y_testmorgan) 

# trainXallUNDERdf.to_csv(split_path+'inhib_train_x_Morgan2comboUNDER.csv', index=False, header=False)
# trainyallUNDERdf.to_csv(split_path+'inhib_train_y_Morgan2comboUNDER.csv', index=False, header=False)
# testydf.to_csv(split_path+'inhib_test_y_Morgan2comboUNDER.csv', index=False, header=False)
# testxdf.to_csv(split_path+'inhib_test_x_Morgan2comboUNDER.csv', index=False, header=False)

trainXallUNDERdf.shape, trainyallUNDERdf.shape, testydf.shape, testxdf.shape

# Combo r=2, ADASYN

In [ ]:
adasyn = ADASYN() 
trainX_allADASYN, trainy_allADASYN = adasyn.fit_resample(X_train_allfeats, y_train)
trainXallADASYNdf = pd.DataFrame(trainX_allADASYN)
trainyallADASYNdf = pd.DataFrame(trainy_allADASYN)
testxdfADASYN = pd.DataFrame(X_test_allfeats)
testydfADASYN = pd.DataFrame(y_testmorgan) 

# trainXallADASYNdf.to_csv(split_path+'inhib_train_x_Morgan2comboADASYN.csv', index=False, header=False)
# trainyallADASYNdf.to_csv(split_path+'inhib_train_y_Morgan2comboADASYN.csv', index=False, header=False)
# testxdfADASYN.to_csv(split_path+'inhib_test_y_Morgan2comboADASYN.csv', index=False, header=False)
# testydfADASYN.to_csv(split_path+'inhib_test_x_Morgan2comboADASYN.csv', index=False, header=False)

trainXallADASYNdf.shape, trainyallADASYNdf.shape, testxdfADASYN.shape, testydfADASYN.shape